In [52]:
with open('input.txt', 'r') as f:
    data = f.read().splitlines()

def parse_instruction(line):
    split_line = [coord.split(',') for coord in line.split(' -> ')]
    parsed_line = [[int(composant) for composant in composant_list] for composant_list in split_line]
    return parsed_line

def get_min_coord(instructions, pourring_point):
    min_x = instructions[0][0][0]
    min_y = instructions[0][0][1]
    for instruction in instructions:
        for coord in instruction:
            if coord[0] < min_x:
                min_x = coord[0]
            if coord[1] < min_y:
                min_y = coord[1]

    if pourring_point[0] < min_x:
        min_x = pourring_point[0]
    if pourring_point[1] < min_y:
        min_y = pourring_point[1]

    return min_x, min_y


def fix_offset(instructions, pourring_point):
    min_x, min_y = get_min_coord(instructions, pourring_point)
    for instruction in instructions:
        for coord in instruction:
            coord[0] -= min_x
            coord[1] -= min_y
    
    pourring_point = (pourring_point[0] - min_x, pourring_point[1] - min_y)

    cave_height = max([max([coord[1] for coord in instruction]) for instruction in instructions]) + 1
    cave_width = max([max([coord[0] for coord in instruction]) for instruction in instructions]) + 1
    
    return instructions, pourring_point, (cave_height, cave_width)
    

def processing_data(data, pourring_point):
    # Parsing instructions
    instructions = [parse_instruction(line) for line in data]

    # Fix offset for every instruction
    instructions, pourring_point, cave_dimensions = fix_offset(instructions, pourring_point)

    return instructions, pourring_point, cave_dimensions

def get_coordinates_from_instruction(instruction):
    rock_coordinates = []
    for i in range(len(instruction)-1) :
        from_coordinate = instruction[i]
        to_coordinate = instruction[i+1]
        # Vertical
        if from_coordinate[0] == to_coordinate[0]:
            range_coord = range(from_coordinate[1], to_coordinate[1] + 1) if from_coordinate[1] < to_coordinate[1] else range(to_coordinate[1], from_coordinate[1] + 1)
            [rock_coordinates.append((from_coordinate[0], y)) for y in range_coord]
        # Horizontal
        elif from_coordinate[1] == to_coordinate[1]:
            range_coord = range(from_coordinate[0], to_coordinate[0] + 1) if from_coordinate[0] < to_coordinate[0] else range(to_coordinate[0], from_coordinate[0] + 1)
            [rock_coordinates.append((x, from_coordinate[1])) for x in range_coord]
        else:
            print('Error')
    return rock_coordinates

def building_cave(instructions, pourring_point, cave_dimensions):
    cave = [[air_symbol for _ in range(cave_dimensions[1])] for _ in range(cave_dimensions[0]+1)] # We make an extra line for the abyss
    for instruction in instructions:
        rock_coordinates = get_coordinates_from_instruction(instruction)
        for rock in rock_coordinates:
            cave[rock[1]][rock[0]] = rock_symbol
        
    # Adding pits to the left and right of the rock
    for i in range(len(cave)):
        cave[i].append(air_symbol)
        cave[i].insert(0, air_symbol)
    
    pourring_point = (pourring_point[0]+1, pourring_point[1]) # We add 1 to the pourring point because we added a column on the left

    cave[pourring_point[1]][pourring_point[0]] = '+'
    return cave, pourring_point

def make_fall(cave, grain_position, grain_number):
    ''' Make the grain fall
    Return the new position of the grain and a boolean to know if the grain fell in the abyss
    '''
    # Check position below
    if cave[grain_position[1]+1][grain_position[0]] == air_symbol:
        new_position = (grain_position[0], grain_position[1]+1)
    # Check diagolal left below
    elif grain_position[0] != 0 and cave[grain_position[1]+1][grain_position[0]-1] == air_symbol:
        new_position = (grain_position[0]-1, grain_position[1]+1)
    # Check diagolal right below
    elif grain_position[0] != len(cave[0])-1 and cave[grain_position[1]+1][grain_position[0]+1] == air_symbol:
        new_position = (grain_position[0]+1, grain_position[1]+1)
    # Can't fall
    else:
        new_position = grain_position
    
    if new_position[1] != len(cave)-1:
        return new_position, False
    else:
        return new_position, True

def pour_sand(cave, pourring_point):
    # Add sand to pourring point
    grain_felt = False
    grain_number = 0

    while not grain_felt:
        grain_position = pourring_point
        grain_number += 1

        new_position, grain_felt = make_fall(cave, grain_position, grain_number)
        while new_position != grain_position:
            cave[grain_position[1]][grain_position[0]] = air_symbol
            grain_position = new_position
            cave[grain_position[1]][grain_position[0]] = sand_symbol
            if grain_felt:
                break
            new_position, grain_felt = make_fall(cave, grain_position, grain_number)
    return grain_number
        

def display_cave(cave):
    for line in cave:
        print('  '.join(line))
        

pourring_point = (500, 0)
air_symbol = '. '
rock_symbol = '#'
sand_symbol = 'O'

instructions, pourring_point, cave_dimensions = processing_data(data, pourring_point)
cave, pourring_point = building_cave(instructions, pourring_point, cave_dimensions)
grain_number = pour_sand(cave, pourring_point)
print(f'We could pour {grain_number-1} sand grain before the next felt into the abyss')
display_cave(cave)


We could pour 1406 sand grain before the next felt into the abyss
.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   . 
.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   . 
.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

In [56]:
with open('input.txt', 'r') as f:
    data = f.read().splitlines()

def parse_instruction(line):
    split_line = [coord.split(',') for coord in line.split(' -> ')]
    parsed_line = [[int(composant) for composant in composant_list] for composant_list in split_line]
    return parsed_line

def get_min_coord(instructions, pourring_point):
    min_x = instructions[0][0][0]
    min_y = instructions[0][0][1]
    for instruction in instructions:
        for coord in instruction:
            if coord[0] < min_x:
                min_x = coord[0]
            if coord[1] < min_y:
                min_y = coord[1]

    if pourring_point[0] < min_x:
        min_x = pourring_point[0]
    if pourring_point[1] < min_y:
        min_y = pourring_point[1]

    return min_x, min_y


def fix_offset(instructions, pourring_point):
    min_x, min_y = get_min_coord(instructions, pourring_point)
    for instruction in instructions:
        for coord in instruction:
            coord[0] -= min_x
            coord[1] -= min_y
    
    pourring_point = (pourring_point[0] - min_x, pourring_point[1] - min_y)

    cave_height = max([max([coord[1] for coord in instruction]) for instruction in instructions]) + 1
    cave_width = max([max([coord[0] for coord in instruction]) for instruction in instructions]) + 1
    
    return instructions, pourring_point, (cave_height, cave_width)
    

def processing_data(data, pourring_point):
    # Parsing instructions
    instructions = [parse_instruction(line) for line in data]

    # Fix offset for every instruction
    instructions, pourring_point, cave_dimensions = fix_offset(instructions, pourring_point)

    return instructions, pourring_point, cave_dimensions

def get_coordinates_from_instruction(instruction):
    rock_coordinates = []
    for i in range(len(instruction)-1) :
        from_coordinate = instruction[i]
        to_coordinate = instruction[i+1]
        # Vertical
        if from_coordinate[0] == to_coordinate[0]:
            range_coord = range(from_coordinate[1], to_coordinate[1] + 1) if from_coordinate[1] < to_coordinate[1] else range(to_coordinate[1], from_coordinate[1] + 1)
            [rock_coordinates.append((from_coordinate[0], y)) for y in range_coord]
        # Horizontal
        elif from_coordinate[1] == to_coordinate[1]:
            range_coord = range(from_coordinate[0], to_coordinate[0] + 1) if from_coordinate[0] < to_coordinate[0] else range(to_coordinate[0], from_coordinate[0] + 1)
            [rock_coordinates.append((x, from_coordinate[1])) for x in range_coord]
        else:
            print('Error')
    return rock_coordinates

def building_cave(instructions, pourring_point, cave_dimensions):
    cave = [[air_symbol for _ in range(cave_dimensions[1])] for _ in range(cave_dimensions[0])] # We make an extra line for the abyss
    for instruction in instructions:
        rock_coordinates = get_coordinates_from_instruction(instruction)
        for rock in rock_coordinates:
            cave[rock[1]][rock[0]] = rock_symbol
        
    # Adding pits to the left and right of the rock
    for i in range(len(cave)):
        cave[i].append(air_symbol)
        cave[i].insert(0, air_symbol)
    
    pourring_point = (pourring_point[0]+1, pourring_point[1]) # We add 1 to the pourring point because we added a column on the left

    cave[pourring_point[1]][pourring_point[0]] = pourring_point_symbol

    # Adding a layer of air then a layer of floor to the bottom of the cave
    cave.append([air_symbol for _ in range(len(cave[0]))])
    cave.append([rock_symbol for _ in range(len(cave[0]))])
    return cave, pourring_point

def make_fall(cave, grain_position):
    ''' Make the grain fall
    Return the new position of the grain and a boolean to know if the grain fell in the abyss
    '''
    # Check position below
    if cave[grain_position[1]+1][grain_position[0]] == air_symbol:
        new_position = (grain_position[0], grain_position[1]+1)
    # Check diagolal left below
    elif grain_position[0] != 0 and cave[grain_position[1]+1][grain_position[0]-1] == air_symbol:
        new_position = (grain_position[0]-1, grain_position[1]+1)
    # Check diagolal right below
    elif grain_position[0] != len(cave[0])-1 and cave[grain_position[1]+1][grain_position[0]+1] == air_symbol:
        new_position = (grain_position[0]+1, grain_position[1]+1)
    # Can't fall
    else:
        new_position = grain_position
    return new_position

def add_column(cave, side):
    if side == 'left':
        for i in range(len(cave)):
            if i == len(cave)-1:
                cave[i].insert(0, rock_symbol)
            else:
                cave[i].insert(0, air_symbol)
    elif side == 'right':
        for i in range(len(cave)):
            if i == len(cave)-1:
                cave[i].append(rock_symbol)
            else:
                cave[i].append(air_symbol)
    return cave


def pour_sand(cave, pourring_point):
    # Add sand to pourring point
    sand_filled = False
    grain_number = 0

    while not sand_filled:
        grain_position = pourring_point
        grain_number += 1

        new_position = make_fall(cave, grain_position)
        while new_position != grain_position:
            # print(new_position)
            cave[grain_position[1]][grain_position[0]] = air_symbol
            grain_position = new_position
            cave[grain_position[1]][grain_position[0]] = sand_symbol
            new_position = make_fall(cave, grain_position)

            # check if grain is on the left side of the cave
            if grain_position[0] == 0:
                cave = add_column(cave, 'left')
                pourring_point = (pourring_point[0]+1, pourring_point[1])
                grain_position = (grain_position[0]+1, grain_position[1])
                new_position = (new_position[0]+1, new_position[1])
            
            # check if grain is on the right side of the cave
            if grain_position[0] == len(cave[0])-1:
                cave = add_column(cave, 'right')

            # Redraw pourring point
            cave[pourring_point[1]][pourring_point[0]] = pourring_point_symbol

        # check if the grain is stucking the pourring point
        if new_position == pourring_point:
            # print(f'There was {grain_number-1} sand grain before stucking the pourring point')  
            sand_filled = True
            break

    return cave, grain_number
        

def display_cave(cave):
    for line in cave:
        print(''.join(line))
        

air_symbol = ' . '
rock_symbol = '# '
sand_symbol = 'O'
pourring_point_symbol = '+'
pourring_point = (500, 0)

instructions, pourring_point, cave_dimensions = processing_data(data, pourring_point)
cave, pourring_point = building_cave(instructions, pourring_point, cave_dimensions)
cave, grain_number = pour_sand(cave, pourring_point)
print(f'There was {grain_number} sand grain before stucking the pourring point')
display_cave(cave)

There was 20870 sand grain before stucking the pourring point
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .+ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .OOO . . . . . . . . . . . . . . . . . . . 